In [2]:
import numpy as np
import pandas as pd
import streamlit as st
from loaders import get_datetime_cols


In [3]:
sheet_url = st.secrets["private_gsheets_url"]

In [4]:
from connections import get_google_conn
import streamlit as st
import pandas as pd

gc = get_google_conn()

# Uses st.cache to only rerun when the query changes or after 10 min.
def get_worksheet(url, sheet_num=0):
    worksheet = gc.open_by_url(url).get_worksheet(sheet_num)
    df = pd.DataFrame(worksheet.get_all_values())
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)
    df.columns = [col.lower() for col in df.columns]
    return df

In [5]:
cheki_df = get_datetime_cols(get_worksheet(sheet_url, 0))
person_df = get_worksheet(sheet_url, 1)
#venue_df = get_worksheet(sheet_url, 2)

In [6]:
person_df

,entity,name1,group1,name2,group2,name3,group3,name4,group4,sns
1,勝月蘭丸@1682,勝月蘭丸,1682,,,,,,,
2,承和 薫@1682,承和 薫,1682,,,,,,,
3,月見夜零@1682,月見夜零,1682,,,,,,,
4,神山おだし@1682,神山おだし,1682,,,,,,,
5,天野ひろこ@.BPM,天野ひろこ,.BPM,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
580,,,,,,,,,,
581,,,,,,,,,,
582,,,,,,,,,,
583,,,,,,,,,,https://twitter.com/Polalight_hina


In [7]:
from streamlit_app import group_cheki_by_name, get_datetime_cols
dated_cheki_df = get_datetime_cols(cheki_df)

names_df = group_cheki_by_name(dated_cheki_df)

In [8]:
names_df

,cheki_id,date,person,location,year,month,name,group,n_shown
0,1,2013-05-05,矢島舞美@℃-ute,Bellesalle Shibuya Garden,2013,5,矢島舞美,℃-ute,5
1,1,2013-05-05,中島早貴@℃-ute,Bellesalle Shibuya Garden,2013,5,中島早貴,℃-ute,5
2,1,2013-05-05,鈴木愛理@℃-ute,Bellesalle Shibuya Garden,2013,5,鈴木愛理,℃-ute,5
3,1,2013-05-05,岡井千聖@℃-ute,Bellesalle Shibuya Garden,2013,5,岡井千聖,℃-ute,5
4,1,2013-05-05,萩原舞@℃-ute,Bellesalle Shibuya Garden,2013,5,萩原舞,℃-ute,5
...,...,...,...,...,...,...,...,...,...
6281,5866,2022-07-31,日向なの@たまご姫【たまプリ】,Gotanda G3,2022,7,日向なの,たまご姫【たまプリ】,2
6282,5867,2022-07-31,日向なの@たまご姫【たまプリ】,Gotanda G3,2022,7,日向なの,たまご姫【たまプリ】,1
6283,5868,2022-07-31,天音ゆめ@たまご姫【たまプリ】,Gotanda G3,2022,7,天音ゆめ,たまご姫【たまプリ】,1
6284,5869,2022-07-31,七瀬千夏@たまご姫【たまプリ】,Gotanda G3,2022,7,七瀬千夏,たまご姫【たまプリ】,1


In [9]:
groupby_select = ["name"]
sort_level = 1
n_shown_columns = sorted(names_df.n_shown.unique())


In [10]:
df = (
    names_df.groupby(["name"] + ["n_shown"])["person"]
    .count()
    .sort_index(level=1)
    .reset_index()
)

In [11]:
df

,name,n_shown,person
0,AIRI,1,1
1,Aarie,1,1
2,Ale 愛礼,1,4
3,Ally,1,7
4,Alyssaありさ,1,5
...,...,...,...
566,SHUN,8,1
567,Sally,8,1
568,Stella,8,1
569,Vivi,8,1


In [12]:
df = (
    df.pivot(index="name", columns="n_shown", values="person")
    .fillna(0)
    .astype(int)
)


In [13]:
df

n_shown,1,2,3,4,5,6,7,8
name,,,,,,,,
AIRI,1,0,0,0,0,0,0,0
Aarie,1,0,0,0,0,0,0,0
Ale 愛礼,4,0,0,0,0,0,0,0
Ally,7,2,0,1,4,0,0,1
Alyssaありさ,5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
黒崎かのん,1,0,0,0,0,0,0,0
黒木結愛,2,0,0,0,0,0,0,0
黒瀬瑠衣,25,1,0,0,0,0,0,0


In [14]:
df["total"] = df.sum(axis=1)
df = df[["total"] + n_shown_columns]
# df = df.rename(columns={"person": "count"})
df = df.sort_values(
    by=["total"] + n_shown_columns, ascending=False
).reset_index()

In [15]:
df

n_shown,name,total,1,2,3,4,5,6,7,8
0,Suzy,1045,978,36,21,5,5,0,0,0
1,恵深あむ,826,776,39,2,2,5,2,0,0
2,七瀬千夏,316,276,9,19,1,0,3,8,0
3,瀬乃悠月,273,244,23,3,0,2,1,0,0
4,天音ゆめ,237,208,7,1,9,1,3,8,0
...,...,...,...,...,...,...,...,...,...,...
408,矢島舞美,1,0,0,0,0,1,0,0,0
409,福田 結花梨,1,0,0,0,0,1,0,0,0
410,船戸ゆきの,1,0,0,0,0,1,0,0,0
411,萩原舞,1,0,0,0,0,1,0,0,0


In [16]:
dated_cheki_df

,note,location,date,name1,name2,name3,name4,name5,name6,name7,name8,month,year
1,,Bellesalle Shibuya Garden,2013-05-05,矢島舞美@℃-ute,中島早貴@℃-ute,鈴木愛理@℃-ute,岡井千聖@℃-ute,萩原舞@℃-ute,,,,5,2013
2,,Best Western Plus Silverdale Beach Hotel,2014-06-21,AIRI@Aither,,,,,,,,6,2014
3,,Best Western Plus Silverdale Beach Hotel,2014-06-21,SAKURA@Aither,,,,,,,,6,2014
4,,Best Western Plus Silverdale Beach Hotel,2014-06-21,YUUKA@Aither,,,,,,,,6,2014
5,,Best Western Plus Silverdale Beach Hotel,2014-06-21,木村美咲@ラストアイドル,,,,,,,,6,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5866,,Gotanda G3,2022-07-31,Suzy,日向なの@たまご姫【たまプリ】,,,,,,,7,2022
5867,,Gotanda G3,2022-07-31,日向なの@たまご姫【たまプリ】,,,,,,,,7,2022
5868,,Gotanda G3,2022-07-31,天音ゆめ@たまご姫【たまプリ】,,,,,,,,7,2022
5869,,Gotanda G3,2022-07-31,七瀬千夏@たまご姫【たまプリ】,,,,,,,,7,2022


In [17]:
dated_subset = dated_cheki_df.query("date >= '2022-01-01' and date < '2022-07-01'")

In [18]:
name_cols = [col for col in dated_subset.columns if "name" in col]
dated_subset["groupset"] = dated_subset[name_cols].apply(lambda x: ','.join(list(set(y for y in x if y != ""))), axis=1)

/tmp/ipykernel_3971/1594611431.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dated_subset["groupset"] = dated_subset[name_cols].apply(lambda x: ','.join(list(set(y for y in x if y != ""))), axis=1)


In [19]:
output = pd.DataFrame(dated_subset.groupby("groupset")['date'].count().sort_values(ascending=False))

In [20]:
output.index.tolist()

['恵深あむ@あむみほ',
 '涼乃みほ@あむみほ',
 'コイヌ フユ@YUENI',
 '松島朱里@YUMEADO EUROPE',
 'Suzy',
 '星名 夢音@メルティハート',
 '侑之りせ@メルティハート',
 'メグ・ピッチ・オリオン@ぷらしゅが',
 '七瀬千夏@たまご姫【たまプリ】',
 '日向なの@たまご姫【たまプリ】',
 '南 歩唯@メルティハート',
 '天音ゆめ@たまご姫【たまプリ】',
 '中谷亜優@たまご姫【たまプリ】',
 '岬あやめ@たまご姫【たまプリ】',
 '桜衣みゆな@たまご姫【たまプリ】',
 '昊乃ひな@Zero Project',
 '椎名まどか@たまご姫【たまプリ】',
 '雨宮れいな@ぷらしゅが',
 '恵深あむ@あむみほ,涼乃みほ@あむみほ',
 '日向さや@ぷらしゅが',
 'きゃりー@プリズムハート',
 '月美うみ@ぷらしゅが',
 '松山ごま@ぷらしゅが',
 'なみきんぐ@ONIONS',
 '東 茉白@Zero Project',
 'Tina@Finally',
 '瀬乃悠月@恒星宇宙',
 'Juri@Finally',
 '黒瀬瑠衣@あっしゅない！',
 '桜衣みゆな@たまご姫【たまプリ】,七瀬千夏@たまご姫【たまプリ】,椎名まどか@たまご姫【たまプリ】,岬あやめ@たまご姫【たまプリ】,日向なの@たまご姫【たまプリ】,天音ゆめ@たまご姫【たまプリ】,中谷亜優@たまご姫【たまプリ】',
 '熊山みゆ@Parasite.Kiss',
 '雅春奈@ユレルランドスケープ',
 'Meg@Finally',
 'Aoi@Finally',
 'Rinka@Finally',
 'のえる@王立アフィリア・スターズ',
 '小上がお@あっしゅない！',
 '瀬名あすか@Parasite.Kiss',
 '未来',
 '對馬 渚@CODE:0000',
 '雨宮れいな@ぷらしゅが,恵深あむ@あむみほ',
 '松島朱里@YUMEADO EUROPE,恵深あむ@あむみほ',
 'Alyssaありさ@Zero Project',
 'りぃ@Zero Project',
 '伽羅このよ@白色のマフィア',
 'Haruna@Finally',
 'Kuyumi@青SHUN学園',
 '花咲杏@あっしゅない！',
 

In [21]:
ones = output.rename(columns={"date": "count"}).query("count == 1").index.tolist()

In [22]:
pd.options.display.max_rows = None

In [23]:
f'1 ({") (".join(ones)})'

"1 (葉月美愛@Freo,恵深あむ@あむみほ) (篠宮奈々美@きいろちゃんず,恵深あむ@あむみほ,なみきんぐ@ONIONS) (千葉 ゆな@きっと、これが世界？,丸瀬こはる@きっと、これが世界？,山下ももな@きっと、これが世界？) (立花咲希@泡沫パーティーズ) (靄@RE:IN) (空士ヒマリ@点染テンセイ少女) (輪廻りん@あげもん！) (秋元悠里@Hey!Mommy!) (石田綾音@SwayEmotionsSlightly) (皇衣みさ@酔生夢死) (猫田 ナツメ@グレーテルは大人にならない) (那華喜 シイナ@点染テンセイ少女。) (辻 海帆@TEARS-ティアーズ -) (SHUN@青SHUN学園) (Sara@QpQueen) (Suzy,日向なの@たまご姫【たまプリ】,星 ゆら音@DopeDrop) (永尾りく@4次元コンパス) (篠宮奈々美@きいろちゃんず,恵深あむ@あむみほ) (蒼井ふう@【eN】) (葉月美愛@Freo) (雪平菜奈@Chou2Precede) (Miyuki@QpQueen) (葉月みな@MAKE) (萩田こころ@iSPY) (花宮まいか@リトスタ！) (花守珠奈@Krays) (残夢 ふうか@ぷらしゅが) (冨十みと@BOCCHI。) (花ノ宮あみな@Asterisk*zero) (色葉和歌子@BooN! BooN! BooooN) (船丘くるみ@Parasite.Kiss) (羽山涼音@あっしゅない！) (美山玲愛@Parasite.Kiss) (近藤亜沙@UPローチ) (葵井るき@ラブアグレッション) (紫王雅琴@ヲドルマヨナカ) (夏川雅@アクアノート) (しゅか@Village in Maier) (櫻城姫奈@百鬼乙女) (星乃成美@PARADISE DISCO) (チョウゼ ツムギ@点染テンセイ少女) (チョウゼ ツムギ@点染テンセイ少女。) (ツキナミ ミツキ@YUENI,コイヌ フユ@YUENI,ハッパ セリ@YUENI,ナウ リアル@YUENI,クルム リカ@YUENI,マユ チャン@YUENI) (愛葉もも@ai*ai) (愛乃もも@Zero Project) (ハレノヒ アカリ@YUENI) (恋羽咲るる@酔生夢死) (恋空夢苺@Parasite.Kiss) (工藤鈴菜@ミラクルファンファーレ！) (嶺井

In [24]:
len(dated_subset)

1476

In [25]:
450/1473

0.3054989816700611

In [26]:
output.rename(columns={"date": "count"}).query("count > 10")

,count
groupset,
恵深あむ@あむみほ,450
涼乃みほ@あむみほ,86
コイヌ フユ@YUENI,81
松島朱里@YUMEADO EUROPE,81
Suzy,71
星名 夢音@メルティハート,58
侑之りせ@メルティハート,46
メグ・ピッチ・オリオン@ぷらしゅが,39
七瀬千夏@たまご姫【たまプリ】,35


In [27]:
pd.options.display.max_rows = None
dated_subset = dated_cheki_df.query("date >= '2022-07-01' and date < '2022-08-01'")
name_cols = [col for col in dated_subset.columns if "name" in col]
dated_subset["groupset"] = dated_subset[name_cols].apply(lambda x: ' & '.join(list(set(y for y in x if y != ""))), axis=1)
output = pd.DataFrame(dated_subset.groupby("groupset")['date'].count().sort_values(ascending=False))
ones = output.rename(columns={"date": "count"}).query("count == 1").index.tolist()

/tmp/ipykernel_3971/1637897328.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dated_subset["groupset"] = dated_subset[name_cols].apply(lambda x: ' & '.join(list(set(y for y in x if y != ""))), axis=1)


In [28]:
f'1 ({") (".join(ones)})'

'1 (椎名まどか@たまご姫【たまプリ】) (桜衣みゆな@たまご姫【たまプリ】) (残夢 ふうか@ぷらしゅが) (榛リサ@オグメンタル) (紫桜倖那@Zero Project) (涼宮ちむ@錬日oTo個色) (瀬名あすか@Parasite.Kiss) (熊山みゆ@Parasite.Kiss) (田村優芽@YORISOERU) (紫王雅琴@ヲドルマヨナカ) (藤川るな@シュアネス) (西村彩有里@ONIONS) (高橋みあ@せろとにん✖️ばくだん) (高橋るい@Girls Crying in Bathroom) (黒崎かのん@ぷらしゅが) (柊木もも@Parasite.Kiss) (Alyssaありさ@Zero Project) (東 茉白@Zero Project) (月野あみ@シュアネス) (Mirei@GANGDEMIC) (Ria@GANGDEMIC) (Suzy & 小上がお@あっしゅない！ & 花咲杏@あっしゅない！) (Suzy & 黒瀬瑠衣@あっしゅない！) (きゃりー@プリズムハート) (くまだかなえ@IZANAGI) (ナウ リアル@YUENI) (ハチぱん@SHOW!ME!機嫌) (ハッパ セリ@YUENI) (一ノ瀬陽天@夜とメルク) (依東杏奈@emiu) (初ちひろ@FiDZ) (小嶋凛@YUMEADO EUROPE & 宮本佑衣@YUMEADO EUROPE & 船戸ゆきの@YUMEADO EUROPE & 松島朱里@YUMEADO EUROPE & 福田 結花梨@YUMEADO EUROPE) (小平明日香@ラブアグレッション) (小町茉莉@ロング・グッドバイ) (岬あやめ@たまご姫【たまプリ】) (日向さや@ぷらしゅが) (旭奈ひかり@Dr.Lolita) (昊乃ひな@Zero Project) (星野理沙@ラブアグレッション) (月美うみ@ぷらしゅが) (𝙎𝙖𝙣𝙖@GiMMiCK)'

In [29]:
output.rename(columns={"date": "count"}).query("count > 1")

,count
groupset,
恵深あむ@あむみほ,47
Suzy,36
涼乃みほ@あむみほ,18
松島朱里@YUMEADO EUROPE,10
恵深あむ@あむみほ & 涼乃みほ@あむみほ,9
メグ・ピッチ・オリオン@ぷらしゅが,9
雅春奈@ユレルランドスケープ,5
なみきんぐ@ONIONS,3
七瀬千夏@たまご姫【たまプリ】,3


In [32]:
output.reset_index().to_csv("cheki-2022-07.csv", index=False)